In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
from sklearn.model_selection import train_test_split

filename = "encoded_data_04_13.csv"
df = pd.read_csv(f"./DataSets/EncodedData/{filename}", sep=",", encoding="UTF-8")
df.head()

,name,height_in_cm,year,goals_for,goals_against,goals,assists,red_cards,yellow_cards,minutes_played,market_value_in_eur,age_at_evaluation,log_market_value_base10,country_of_citizenship_encoded,sub_position_encoded,club_id_encoded,domestic_competition_id_encoded
0,Miroslav Klose,184.0,2012.75,31.0,14.0,11.0,1.0,0.0,6.0,1483.0,4000000.0,35,6.60206,7.191504e+06,5.331868e+06,7.836111e+06,7.182738e+06
1,Miroslav Klose,184.0,2013.25,24.0,15.0,5.0,2.0,0.0,2.0,1102.0,2000000.0,35,6.30103,7.191504e+06,5.331868e+06,7.836111e+06,7.182738e+06
2,Miroslav Klose,184.0,2013.75,14.0,21.0,4.0,2.0,0.0,1.0,950.0,1000000.0,36,6.00000,7.191504e+06,5.331868e+06,7.836111e+06,7.182738e+06
3,Miroslav Klose,184.0,2014.25,24.0,22.0,4.0,3.0,0.0,1.0,1270.0,1000000.0,36,6.00000,7.191504e+06,5.331868e+06,7.836111e+06,7.182738e+06
4,Miroslav Klose,184.0,2014.75,31.0,18.0,4.0,3.0,0.0,2.0,496.0,1000000.0,37,6.00000,7.191504e+06,5.331868e+06,7.836111e+06,7.182738e+06


In [2]:
df = df.sample(n=5000, random_state=42)

In [3]:
# log_market_value_base10
df = df.drop(columns=['name', 'market_value_in_eur'])
# Del dataene inn i treningssett og testsett
train_data, test_data = train_test_split(df, test_size=0.25, random_state=42)  # Juster test_size og random_state etter behov

# Konverter dataene til TabularDataset-objekter
train_data = TabularDataset(train_data)
test_data = TabularDataset(test_data)
train_data.head()



,height_in_cm,year,goals_for,goals_against,goals,assists,red_cards,yellow_cards,minutes_played,age_at_evaluation,log_market_value_base10,country_of_citizenship_encoded,sub_position_encoded,club_id_encoded,domestic_competition_id_encoded
120238,177.0,2023.25,10.0,7.0,0.0,1.0,0.0,1.0,125.0,19,6.602060,3.902287e+06,5.865733e+06,7.953486e+06,2.483243e+06
110501,180.0,2020.75,20.0,15.0,0.0,1.0,0.0,1.0,580.0,19,5.397940,1.912235e+06,3.362303e+06,3.677499e+05,6.786176e+05
77896,179.0,2015.75,3.0,1.0,0.0,0.0,0.0,0.0,120.0,22,7.113943,3.902287e+06,3.650476e+06,3.530455e+06,8.084417e+06
15492,188.0,2016.75,8.0,3.0,0.0,0.0,0.0,0.0,450.0,31,7.176091,8.564591e+06,4.318791e+06,6.743160e+06,2.390848e+06
66588,190.0,2020.25,1.0,12.0,0.0,0.0,0.0,0.0,519.0,29,5.511883,3.559251e+06,5.331868e+06,4.500000e+05,2.118387e+06


In [4]:
from autogluon.tabular import TabularPredictor

# Create and train the model with a specified time limit
predictor = TabularPredictor(label='log_market_value_base10', problem_type="regression", eval_metric="mean_absolute_percentage_error").fit(train_data, presets='high_quality', time_limit=28800)  # This will limit the training to one hour (3600 seconds)

# Evaluate the model on the test set
performance = predictor.evaluate(test_data)
print(performance)


No path specified. Models will be saved in: "AutogluonModels\ag-20240418_220254"
Presets specified: ['high_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 28800 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels\ag-20240418_220254/ds_sub_fit/sub_fit_ho.
Beginning AutoGluon training ... Time limit = 7200s
AutoGluon will save models to "AutogluonModels\ag-20240418_220254/ds_sub_fit/sub_fit_ho"
=================== System Info ===================
AutoGluon Version:  

[1000]	valid_set's l2: 0.0914837	valid_set's mean_absolute_percentage_error: -0.0594576


	-0.0358	 = Validation score   (-mean_absolute_percentage_error)
	7.63s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 4790.99s of the 7192.16s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-0.0361	 = Validation score   (-mean_absolute_percentage_error)
	5.32s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: RandomForestMSE_BAG_L1 ... Training model for up to 4785.6s of the 7186.77s of remaining time.
	-0.038	 = Validation score   (-mean_absolute_percentage_error)
	0.76s	 = Training   runtime
	0.13s	 = Validation runtime
Fitting model: CatBoost_BAG_L1 ... Training model for up to 4784.64s of the 7185.81s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-0.0354	 = Validation score   (-mean_absolute_percentage_error)
	11.26s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model:

[1000]	valid_set's l2: 0.0957999	valid_set's mean_absolute_percentage_error: -0.0592899
[1000]	valid_set's l2: 0.0737959	valid_set's mean_absolute_percentage_error: -0.0526515
[1000]	valid_set's l2: 0.0854494	valid_set's mean_absolute_percentage_error: -0.0531681


	-0.0359	 = Validation score   (-mean_absolute_percentage_error)
	10.99s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: NeuralNetFastAI_r191_BAG_L1 ... Training model for up to 4737.21s of the 7138.38s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
		Import fastai failed. A quick tip is to install via `pip install autogluon.tabular[fastai]==1.0.0`. 
Fitting model: CatBoost_r9_BAG_L1 ... Training model for up to 4737.08s of the 7138.25s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-0.0357	 = Validation score   (-mean_absolute_percentage_error)
	41.97s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: LightGBM_r96_BAG_L1 ... Training model for up to 4695.02s of the 7096.18s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy


...

[1000]	valid_set's l2: 0.0845168	valid_set's mean_absolute_percentage_error: -0.0579036
[1000]	valid_set's l2: 0.0683141	valid_set's mean_absolute_percentage_error: -0.0535169
[1000]	valid_set's l2: 0.084622	valid_set's mean_absolute_percentage_error: -0.0544999
[1000]	valid_set's l2: 0.0789681	valid_set's mean_absolute_percentage_error: -0.0575739
[2000]	valid_set's l2: 0.0751074	valid_set's mean_absolute_percentage_error: -0.0552235
[3000]	valid_set's l2: 0.0748817	valid_set's mean_absolute_percentage_error: -0.0533339
[4000]	valid_set's l2: 0.0753167	valid_set's mean_absolute_percentage_error: -0.0539224
[1000]	valid_set's l2: 0.0815757	valid_set's mean_absolute_percentage_error: -0.0544145
[1000]	valid_set's l2: 0.0784317	valid_set's mean_absolute_percentage_error: -0.0551661
[1000]	valid_set's l2: 0.074164	valid_set's mean_absolute_percentage_error: -0.0551


	-0.0356	 = Validation score   (-mean_absolute_percentage_error)
	19.81s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: XGBoost_r49_BAG_L1 ... Training model for up to 25979.45s of the 25979.44s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-0.0362	 = Validation score   (-mean_absolute_percentage_error)
	10.52s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: CatBoost_r5_BAG_L1 ... Training model for up to 25968.87s of the 25968.85s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
	-0.0352	 = Validation score   (-mean_absolute_percentage_error)
	16.81s	 = Training   runtime
	0.02s	 = Validation runtime
Fitting model: NeuralNetTorch_r87_BAG_L1 ... Training model for up to 25952.0s of the 25951.99s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
		Unable to import dependency t

[1000]	valid_set's l2: 0.0835181	valid_set's mean_absolute_percentage_error: -0.0544119
[1000]	valid_set's l2: 0.0785673	valid_set's mean_absolute_percentage_error: -0.0553188


	-0.0359	 = Validation score   (-mean_absolute_percentage_error)
	9.07s	 = Training   runtime
	0.05s	 = Validation runtime
Fitting model: NeuralNetTorch_r1_BAG_L1 ... Training model for up to 25742.16s of the 25742.15s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
		Unable to import dependency torch
A quick tip is to install via `pip install torch`.
The minimum torch version is currently 2.0.
Fitting model: NeuralNetTorch_r89_BAG_L1 ... Training model for up to 25742.05s of the 25742.04s of remaining time.
	Fitting 8 child models (S1F1 - S1F8) | Fitting with SequentialLocalFoldFittingStrategy
		Unable to import dependency torch
A quick tip is to install via `pip install torch`.
The minimum torch version is currently 2.0.
Fitting model: WeightedEnsemble_L2 ... Training model for up to 2658.09s of the 25741.89s of remaining time.
	Ensemble Weights: {'CatBoost_r69_BAG_L1': 0.267, 'XGBoost_r89_BAG_L1': 0.2, 'LightGBM_r30_BAG_L1':

{'mean_absolute_percentage_error': -0.03333744234728, 'root_mean_squared_error': -0.2650150575119001, 'mean_squared_error': -0.07023298070803571, 'mean_absolute_error': -0.20330485839119575, 'r2': 0.8339171754450809, 'pearsonr': 0.913750116998979, 'median_absolute_error': -0.1564623492876791}


In [7]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2_FULL,-0.033337,NaN,mean_absolute_percentage_error,0.096253,NaN,5.430458,0.002990,NaN,0.645138,2,True,130
1,LightGBM_r30_BAG_L1_FULL,-0.033437,NaN,mean_absolute_percentage_error,0.051090,NaN,0.874967,0.051090,NaN,0.874967,1,True,107
2,CatBoost_r12_BAG_L1_FULL,-0.033438,NaN,mean_absolute_percentage_error,0.004494,NaN,1.688933,0.004494,NaN,1.688933,1,True,123
3,CatBoost_r86_BAG_L1_FULL,-0.033449,NaN,mean_absolute_percentage_error,0.007487,NaN,1.918976,0.007487,NaN,1.918976,1,True,98
4,CatBoost_r167_BAG_L1_FULL,-0.033499,NaN,mean_absolute_percentage_error,0.004579,NaN,0.793416,0.004579,NaN,0.793416,1,True,95
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,XGBoost_r98_BAG_L1,NaN,-0.037176,mean_absolute_percentage_error,NaN,0.097966,86.716743,NaN,0.097966,86.716743,1,False,31
126,LightGBMLarge_BAG_L1,NaN,-0.037219,mean_absolute_percentage_error,NaN,0.041363,8.550448,NaN,0.041363,8.550448,1,False,9
127,XGBoost_r34_BAG_L1,NaN,-0.037660,mean_absolute_percentage_error,NaN,0.066364,20.925748,NaN,0.066364,20.925748,1,False,63
128,LightGBM_r196_BAG_L1,NaN,-0.037689,mean_absolute_percentage_error,NaN,0.094961,22.956353,NaN,0.094961,22.956353,1,False,28


In [11]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import numpy as np

predictions_train = predictor.predict(train_data)
predictions_test = predictor.predict(test_data)

# Calculate MAE and MAPE on the training data
mae_train = mean_absolute_error(np.power(10, train_data['log_market_value_base10']), predictions_train)
mape_train = 100 * mean_absolute_percentage_error(np.power(10, train_data['log_market_value_base10']), predictions_train)
    
print("Training Set Scores:")
print("Mean Absolute Error (MAE):", round(mae_train / 1000000, 2), "M")
print("Mean Absolute Percentage Error (MAPE):", round(mape_train, 2), "%")
    
# Calculate MAE and MAPE on the test data
mae = mean_absolute_error(np.power(10, test_data['log_market_value_base10']), predictions_test)
mape = 100 * mean_absolute_percentage_error(np.power(10, test_data['log_market_value_base10']), predictions_test)
    
print("\nTest Set Scores:")
print("Mean Absolute Error (MAE):", round(mae / 1000000, 2), "M")
print("Mean Absolute Percentage Error (MAPE):", round(mape, 2), "%")

Training Set Scores:
Mean Absolute Error (MAE): 4.64 M
Mean Absolute Percentage Error (MAPE): 100.0 %

Test Set Scores:
Mean Absolute Error (MAE): 4.72 M
Mean Absolute Percentage Error (MAPE): 100.0 %
